In [2]:
import numpy as np
import scipy
import pickle
import copy

#### 4th Cumulant, Kurtosis at equilibrium

### Need to compute <rr|T_1^t|ll>, <r|T_1^t|l>, <r,r|T_2^t|l,l>

### Define states ll and rr

def mapping(x1,x2):
    x = min(x1,x2)
    y = x1+x2-x
    return int(np.around(x+(y-1)*(y-2)/2,0))

def ket1(x1,x2,L):
    x = min(x1,x2)
    y = x1+x2-x
    label = int(np.around(x+(y-1)*(y-2)/2,0))
    temp = np.eye(int(np.around(L*(L-1)/2,0)))
    return temp[label-1]

def ket2(x,L):
    return np.eye(L)[x-1]

def RR(L):
    dim1 = int(np.around(L*(L-1)/2,0))

    RR = np.zeros((dim1))
    for x2 in range(L//2+1,L+1):
        for x1 in range(L//2+1,x2):
            RR += ket1(x1,x2,L)
    return RR

def LL(L):
    dim1 = int(np.around(L*(L-1)/2,0))

    ll = np.zeros((dim1))
    for x2 in range(2,L//2+1):
        for x1 in range(1,x2):
            ll += ket1(x1,x2,L)
    return ll

def TT(L):
    ### this is supposed to be the GS, i.e., unbiased sum over all magnon configurations
    dim1 = int(np.around(L*(L-1)/2,0))

    tt = np.zeros((dim1))
    for x2 in range(2,L+1):
        for x1 in range(1,x2):
            tt += ket1(x1,x2,L)
    return tt

def R_R(L):
    r_ = np.zeros((L))
    for x in range(L//2+1,L+1):
        r_ += ket2(x,L)
    r_r = np.kron(r_,r_)
    return np.reshape(r_r,(L**2))

def L_L(L):
    l_ = np.zeros((L))
    for x in range(1,L//2+1):
        l_ += ket2(x,L)
    l_l = np.kron(l_,l_)
    return np.reshape(l_l,(L**2))

def T_T(L):
    t_ = np.zeros((L))
    for x in range(1,L+1):
        t_ += ket2(x,L)
    t_t = np.kron(t_,t_)
    return np.reshape(t_t,(L**2))

def R_(L):
    r_ = np.zeros((L))
    for x in range(L//2+1,L+1):
        r_ += ket2(x,L)
    return r_

def L_(L):
    l_ = np.zeros((L))
    for x in range(1,L//2+1):
        l_ += ket2(x,L)
    return l_

def T_(L):
    t_ = np.zeros((L))
    for x in range(1,L+1):
        t_ += ket2(x,L)
    return t_

### Define time evolution

### One magnon evolution

def SingleRailOneMagnon_odd_update(L,state):
    wstate = np.sqrt(0.5)*(np.array((1,1)))
    gate2 = np.outer(wstate,wstate)
    newstate = np.zeros((L))
    for j in range(0,L-1,2):
        statejjp1 = state[j:j+2]
        newstate[j:j+2] += np.dot(gate2,statejjp1)
    return newstate

def SingleRailOneMagnon_even_update(L,state):
    wstate = np.sqrt(0.5)*(np.array((1,1)))
    gate2 = np.outer(wstate,wstate)
    newstate = np.zeros((L))
    for j in range(1,L-1,2):
        x = j+1
        statejjp1 = state[j:j+2]
        newstate[j:j+2] += np.dot(gate2,statejjp1)
    ## deal with sites 1 and L 
    newstate[0] += state[0]
    newstate[L-1] += state[L-1]
    return newstate
    
### 1 + 1 magnon evolution

def TwoRail_odd_update(L,state,lam):
    state = np.reshape(state,(L,L))
    tempstate = copy.deepcopy(state)
    wstate = np.sqrt(0.5)*(np.array((1,1)))
    vstate = np.sqrt(0.5)*np.array((1,-1))
    gate = np.outer(wstate,wstate)
    for y in range(L):
        for i in range(0,L-1,2):
            x = i+1
            state[x-1:x+1,y] = np.dot(gate,state[x-1:x+1,y])
    for y in range(L):
        for i in range(0,L-2,2):
            x = i+1
            state[y,x-1:x+1] = np.dot(gate,state[y,x-1:x+1])
    
    ### add the interactions between rails, use the state stored in tempstate
    vgate = np.outer(vstate,vstate)
    tempstate1 = copy.deepcopy(tempstate)
    newstate1 = np.zeros((L,L))
    ### The 1 - 2 interaction
    for i in range(0,L-1,2): # position of magnon in rail 1,2
        x = i+1
        newstate1[x-1:x+1,x-1:x+1] = np.reshape(np.dot(np.kron(vgate,vgate),np.reshape(tempstate1[x-1:x+1,x-1:x+1],(4))),(2,2))
    
    state = state + lam*newstate1
    return np.reshape(state,(L**2))

def TwoRail_even_update(L,state,lam):
    ### write the Even-Odd bond update step.
    state = np.reshape(state,(L,L))
    tempstate = copy.deepcopy(state)
    wstate = np.sqrt(0.5)*(np.array((1,1)))
    vstate = np.sqrt(0.5)*np.array((1,-1))
    wgate = np.outer(wstate,wstate)
    vgate = np.outer(vstate,vstate)
    for y in range(L):
        for i in range(1,L-1,2):
            x = i+1
            state[x-1:x+1,y] = np.dot(wgate,state[x-1:x+1,y])
    for y in range(L):
        for i in range(1,L-1,2):
            x = i+1
            state[y,x-1:x+1] = np.dot(wgate,state[y,x-1:x+1])

    ### add the interactions between rails, use the state stored in tempstate
    tempstate1 = copy.deepcopy(tempstate)
    newstate1 = np.zeros((L,L))
    ### The 1 - 2 interaction 
    for i in range(1,L-1,2): # position of magnon in rail 1,2
        x = i+1
        newstate1[x-1:x+1,x-1:x+1] = np.reshape(np.dot(np.kron(vgate,vgate),np.reshape(tempstate1[x-1:x+1,x-1:x+1],(4))),(2,2))

    state = state + lam*newstate1
    return np.reshape(state,(L**2))

### Two magnon evolution

def SingleRailTwoMagnon_odd_update(L,state): ### THIS APPEARS TO WORK!
    dim1 = int(np.around(L*(L-1)/2,0))
    state = np.reshape(state,(dim1)) # |state> = sum_{s1,s2} |s1> \otimes |s2>
    tempstate = copy.deepcopy(state)
    wstate = np.sqrt(0.5)*(np.array((1,1)))
    gate1 = np.outer(np.kron(wstate,wstate),np.kron(wstate,wstate))
    gate2 = np.outer(wstate,wstate)
            
    ## consider magnons in different odd-even pairs first
    state1 = tempstate
    newstate1 = np.zeros((dim1))
    for j in range(2,L-1,2):
        x2 = j+1
        for k in range(0,j,2):
            x1 = k+1
            x1x2 = mapping(x1,x2)-1
            x1p1x2 = mapping(x1+1,x2)-1
            x1x2p1 = mapping(x1,x2+1)-1
            x1p1x2p1 = mapping(x1+1,x2+1)-1
            state1_kkp1_jjp1 = np.array((state1[x1x2],state1[x1x2p1],state1[x1p1x2],state1[x1p1x2p1]))
            newstate1_kkp1_jjp1 = np.dot(gate1,state1_kkp1_jjp1)
            newstate1[x1x2] = newstate1_kkp1_jjp1[0]
            newstate1[x1x2p1] = newstate1_kkp1_jjp1[1]
            newstate1[x1p1x2] = newstate1_kkp1_jjp1[2]
            newstate1[x1p1x2p1] = newstate1_kkp1_jjp1[3]
    ### now consider magnons in the same odd-even pair
    for j in range(0,L-1,2):
        x1 = j+1
        x1x2 = mapping(x1,x1+1)-1
        newstate1[x1x2] = state1[x1x2]
    
    return np.reshape(newstate1,(dim1))

def SingleRailTwoMagnon_even_update(L,state):
    dim1 = int(np.around(L*(L-1)/2,0))
    state = np.reshape(state,(dim1)) # |state> = sum_{s1,s2} |s1> \otimes |s2>
    tempstate = copy.deepcopy(state)
    wstate = np.sqrt(0.5)*(np.array((1,1)))
    gate1 = np.outer(np.kron(wstate,wstate),np.kron(wstate,wstate))
    gate2 = np.outer(wstate,wstate)
        
    ## consider magnons in different even-odd pairs first
    state1 = state
    newstate1 = np.zeros((dim1))
    for j in range(3,L-1,2):
        x2 = j+1
        for k in range(1,j,2):
            x1 = k+1
            x1x2 = mapping(x1,x2)-1
            x1p1x2 = mapping(x1+1,x2)-1
            x1x2p1 = mapping(x1,x2+1)-1
            x1p1x2p1 = mapping(x1+1,x2+1)-1
            state1_kkp1_jjp1 = np.array((state1[x1x2],state1[x1x2p1],state1[x1p1x2],state1[x1p1x2p1]))
            gate = np.outer(np.kron(wstate,wstate),np.kron(wstate,wstate))
            newstate1_kkp1_jjp1 = np.dot(gate,state1_kkp1_jjp1)
            newstate1[x1x2] = newstate1_kkp1_jjp1[0]
            newstate1[x1x2p1] = newstate1_kkp1_jjp1[1]
            newstate1[x1p1x2] = newstate1_kkp1_jjp1[2]
            newstate1[x1p1x2p1] = newstate1_kkp1_jjp1[3]
    ### now consider one magnon at position 1 and one in the bulk
    for j in range(1,L-1,2):
        x2 = j+1
        x1 = 1
        x1x2 = mapping(x1,x2)-1
        x1x2p1 = mapping(x1,x2+1)-1
        state1_1_jjp1 = np.array((state1[x1x2],state1[x1x2p1]))
        gate = np.outer(np.kron(np.array((1)),wstate),np.kron(np.array((1)),wstate))
        newstate1_1_jjp1 = np.dot(gate,state1_1_jjp1)
        newstate1[x1x2] = newstate1_1_jjp1[0]
        newstate1[x1x2p1] = newstate1_1_jjp1[1]
            
    ### now consider one magnon at position L and one in the bulk
    for j in range(1,L-1,2):
        x1 = j+1
        x2 = L
        x1x2 = mapping(x1,x2)-1
        x1p1x2 = mapping(x1+1,x2)-1
        state1_jjp1_L = np.array((state1[x1x2],state1[x1p1x2]))
        gate = np.outer(np.kron(wstate,np.array((1))),np.kron(wstate,np.array((1))))
        newstate1_jjp1_L = np.dot(gate,state1_jjp1_L)
        newstate1[x1x2] = newstate1_jjp1_L[0]
        newstate1[x1p1x2] = newstate1_jjp1_L[1]
            
    ### now consider one magnon at site 1 and one at site L
    x1x2 = mapping(1,L)-1
    state1_1_L = np.array((state1[x1x2]))
    newstate1[x1x2] = state1_1_L
        
    ### now consider magnons in the same odd-even pair
    for j in range(1,L-1,2):
        x1 = j+1
        x1x2 = mapping(x1,x1+1)-1
        newstate1[x1x2] = state1[x1x2]
        
    newstate1
        
    return np.reshape(newstate1,(dim1))

### Run simulation and gather data

def TwoMagnonSim(L,tmax,lam):
    dim1 = int(np.around(L*(L-1)/2,0))
    rr = copy.deepcopy(rr0)
    ll = copy.deepcopy(ll0)
    tt = copy.deepcopy(tt0)
    r_ = copy.deepcopy(r_0)
    l_ = copy.deepcopy(l_0)
    t_ = copy.deepcopy(t_0)
    r_r = copy.deepcopy(r_r0)
    l_l = copy.deepcopy(l_l0)
    t_t = copy.deepcopy(t_t0)
    
    results = {'L': L, 'lam': lam, 'C4': [], 'KurtSEP': [], 'KurtProxy': [], 'llrr': [], 'lr': [], 'l_lr_r': [], 'norm1': [], 'norm2': [], 'norm3': [], 'll': [], 'l': [], 'l_l': []}
    results['llrr'].append(np.dot(ll,rr))
    results['lr'].append(np.dot(l_,r_))
    results['l_lr_r'].append(np.dot(l_l,r_r))
    results['C4'].append((3/2)*results['llrr'][-1]-(3/4)*results['l_lr_r'][-1])
    results['norm1'].append(np.dot(l_,t_))
    results['norm2'].append(np.dot(ll,tt))
    results['norm3'].append(np.dot(l_l,t_t))
    results['ll'].append(ll)
    results['l'].append(l_)
    results['l_l'].append(l_l)    
    
    t = 0
    ### Evolve the state
    while t < tmax:
        ll = SingleRailTwoMagnon_even_update(L,ll)
        ll = SingleRailTwoMagnon_odd_update(L,ll)
        l_ = SingleRailOneMagnon_even_update(L,l_)
        l_ = SingleRailOneMagnon_odd_update(L,l_)
        l_l = TwoRail_even_update(L,l_l,lam)
        l_l = TwoRail_odd_update(L,l_l,lam)        
        results['llrr'].append(np.dot(ll,rr))
        results['lr'].append(np.dot(l_,r_))
        results['l_lr_r'].append(np.dot(l_l,r_r))
        results['KurtSEP'].append(6*np.dot(ll,rr)/np.dot(l_,r_)**2)
        results['KurtProxy'].append(6*np.dot(ll,rr)/np.dot(l_l,r_r))
        results['C4'].append((3/2)*results['llrr'][-1]-(3/4)*results['l_lr_r'][-1])
        results['norm1'].append(np.dot(l_,t_))
        results['norm2'].append(np.dot(ll,tt))
        results['norm3'].append(np.dot(l_l,t_t))
        results['ll'].append(ll)
        results['l'].append(l_)
        results['l_l'].append(l_l)
        
        results_dic[lam] = results
        with open(filename, 'wb') as f:
            pickle.dump(results_dic,f)
        t += 1
    
    return results

L = 120
tmax = 120

dimlist = [0.845,0.85,0.875,0.9,0.95,1,1.5,2,3,4]
lamlist = [1/(4*d**4-1) for d in dimlist]
#lamlist = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99]
results_dic = {}

filename = 'Kurtosis_C4_SEP_vs_Stoch_%d.pkl'%L

rr0 = RR(L)
ll0 = LL(L)
tt0 = TT(L)
r_0 = R_(L)
l_0 = L_(L)
t_0 = T_(L)
r_r0 = R_R(L)
l_l0 = L_L(L)
t_t0 = T_T(L)

for lam in lamlist:
    TwoMagnonSim(L,tmax,lam)